<a href="https://colab.research.google.com/github/thanospapastef/Master-thesis---Transfer-Learning/blob/main/Master_thesis_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries
!pip3 install opencv-python
import cv2
import math
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np
from keras.utils import np_utils
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models
from sklearn.metrics import accuracy_score
import os
from scipy import stats as s
from keras.callbacks import ModelCheckpoint
import pickle

In [ ]:
#downloading and extracting UCF-101 dataset
!wget -nc --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
!unrar e UCF101.rar Videos/

In [ ]:
#creating train and test datasets according to authors of UCF-101 (i.e. appropriate dataset split) 
f = open("trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.shape

f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.shape

train_video_tag = []

for i in range(train.shape[0]):
  train_video_tag.append(train['video_name'][i].split('/')[0])

train['tag'] = train_video_tag

test_video_tag = []

for i in range(test.shape[0]):
  test_video_tag.append(test['video_name'][i].split('/')[0])

for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('Videos/'+videoFile.split(' ')[0].split('/')[1])
    frameRate = cap.get(5)

    while(cap.isOpened()):
        frameId = cap.get(1)
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):

# storing the frames in a new folder named train_1
            filename ='train_1/' +
            videoFile.split('/')[1].split(' ')[0]+
            "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
images = glob("train_1/*.jpg")
train_image = []
train_class = []

for i in tqdm(range(len(images))):

    train_image.append(images[i].split('/')[1])

    train_class.append(images[i].split('/')[1].split('_')[1])

train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

train_data.to_csv('Videos/train_new.csv',header=True, index=False)

train = pd.read_csv('Videos/train_new.csv')

images = glob("train_1/*.jpg")
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)
X.shape

y = train['class']

X_train, X_test, y_train, y_test = train_test_split(X, y,
random_state=42, test_size=0.2, stratify = y)

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)
y = pd.get_dummies(y)

shape = (224, 224, 3)

base_model_1 = tf.keras.applications.VGG16(weights='imagenet',
include_top=False, input_shape=shape)

base_model_2 = tf.keras.applications.InceptionV3(weights='imagenet',
include_top=False, input_shape=shape)

base_model_3 = tf.keras.applications.Xception(weights='imagenet',
include_top=False, input_shape=shape)

X_train_VGG16 = base_model_1.predict(X_train)
X_train_VGG16.shape

X_test_VGG16 = base_model_1.predict(X_test)
X_test_VGG16.shape

X_train_InceptionV3 = base_model_2.predict(X_train)
X_train_InceptionV3.shape

X_test_InceptionV3 = base_model_2.predict(X_test)
X_test_InceptionV3.shape

X_train_Xception = base_model_3.predict(X_train)
X_train_Xception.shape

X_test_Xception = base_model_3.predict(X_test)
X_test_Xception.shape

X_train_VGG16 = X_train_VGG16.reshape(59075, 7*7*512)
X_test_VGG16 = X_test_VGG16.reshape(14769, 7*7*512)

X_train_InceptionV3 = X_train_InceptionV3.reshape(59075, 5*5*2048)
X_test_InceptionV3 = X_test_InceptionV3.reshape(14769, 5*5*2048)

X_train_Xception = X_train_Xception.reshape(59075, 7*7*2048)
X_test_Xception = X_test_Xception.reshape(14769, 7*7*2048)

max = X_train_VGG16.max()
X_train_VGG16 = X_train_VGG16/max
X_test_VGG16 = X_test_VGG16/max

max = X_train_InceptionV3.max()
X_train_InceptionV3 = X_train_InceptionV3/max
X_test_InceptionV3 = X_test_InceptionV3/max

max = X_train_Xception.max()
X_train_Xception = X_train_Xception/max
X_test_Xception = X_test_Xception/max

In [ ]:
model_1 = Sequential()
model_1.add(Dense(1024, activation='relu', input_shape=(25088,)))
model_1.add(Dropout(0.5))
model_1.add(Dense(512, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(256, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(128, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(101, activation='softmax'))

model_2 = Sequential()
model_2.add(Dense(1024, activation='relu', input_shape=(51200,)))
model_2.add(Dropout(0.5))
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(256, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(128, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(101, activation='softmax'))

model_3 = Sequential()
model_3.add(Dense(1024, activation='relu', input_shape=(100352,)))
model_3.add(Dropout(0.5))
model_3.add(Dense(512, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(256, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(128, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(101, activation='softmax'))

model_1.compile(loss='categorical_crossentropy',optimizer='Adam',
metrics=['accuracy'])

model_2.compile(loss='categorical_crossentropy',optimizer='Adam',
metrics=['accuracy'])

model_3.compile(loss='categorical_crossentropy',optimizer='Adam',
metrics=['accuracy'])

In [ ]:
mcp_save_1 = ModelCheckpoint('VGG16.hdf5', save_best_only=True,
monitor='val_loss', mode='min')

# training the model
model_training_history_1 = model_1.fit(X_train_VGG16, y_train, epochs=100,
validation_data=(X_test_VGG16, y_test), callbacks = mcp_save_1,
batch_size=128)

def plot_metric(metric_name_1, metric_name_2, plot_name):

    # Get Metric values using metric names as identifiers
    metric_value_1 = model_training_history_1.history[metric_name_1]
    metric_value_2 = model_training_history_1.history[metric_name_2]

    # Constructing a range object which will be used as time
    epochs = range(len(metric_value_1))

    # Plotting the Graph
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Adding title to the plot
    plt.title(str(plot_name))

    # Adding legend to the plot
    plt.legend()

plot_metric('loss', 'val_loss', 'Total Loss vs Total Validation Loss')
plot_metric('accuracy', 'val_accuracy', 'Total Accuracy vs
Total Validation Accuracy')

In [ ]:
mcp_save_2 = ModelCheckpoint('InceptionV3.hdf5', save_best_only=True,
monitor='val_loss', mode='min')

# training the model
model_training_history_2 = model_2.fit(X_train_InceptionV3, y_train, epochs=100,
validation_data=(X_test_InceptionV3, y_test), callbacks = mcp_save_2,
batch_size=128)

def plot_metric(metric_name_1, metric_name_2, plot_name):

    # Get Metric values using metric names as identifiers
    metric_value_1 = model_training_history_2.history[metric_name_1]
    metric_value_2 = model_training_history_2.history[metric_name_2]

    # Constructing a range object which will be used as time
    epochs = range(len(metric_value_1))

    # Plotting the Graph
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Adding title to the plot
    plt.title(str(plot_name))

    # Adding legend to the plot
    plt.legend()
plot_metric('loss', 'val_loss', 'Total Loss vs Total Validation Loss')
plot_metric('accuracy', 'val_accuracy', 'Total Accuracy vs
Total Validation Accuracy')

In [ ]:
mcp_save_3 = ModelCheckpoint('Xception.hdf5', save_best_only=True,
monitor='val_loss', mode='min')

# training the model
model_training_history_3 = model_3.fit(X_train_Xception, y_train, epochs=100,
validation_data=(X_test_Xception, y_test), callbacks = mcp_save_3,
batch_size=128)

def plot_metric(metric_name_1, metric_name_2, plot_name):

    # Get Metric values using metric names as identifiers
    metric_value_1 = model_training_history_3.history[metric_name_1]
    metric_value_2 = model_training_history_3.history[metric_name_2]

    # Constructing a range object which will be used as time
    epochs = range(len(metric_value_1))

    # Plotting the Graph
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Adding title to the plot
    plt.title(str(plot_name))

    # Adding legend to the plot
    plt.legend()
plot_metric('loss', 'val_loss', 'Total Loss vs Total Validation Loss')
plot_metric('accuracy', 'val_accuracy', 'Total Accuracy vs
Total Validation Accuracy')

In [ ]:
predict_1 = []
predict_2 = []
predict_3 = []
actual = []

for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('Videos/'+videoFile.split(' ')[0].split('/')[1])
    frameRate = cap.get(5) #frame rate

    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()

        if (ret != True):
          break

        if (frameId % math.floor(frameRate) == 0):
          filename ='temp/' + "_frame%d.jpg" % count;count+=1
          cv2.imwrite(filename, frame)

    cap.release()

    # reading all the frames from temp folder
    images = glob("temp/*.jpg")

    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)

    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)

    # extracting features using pre-trained model
    prediction_images_1 = base_model_1.predict(prediction_images)
    prediction_images_2 = base_model_2.predict(prediction_images)
    prediction_images_3 = base_model_3.predict(prediction_images)

    # converting features in one dimensional array
    # do the same for each model
    prediction_images_1 = prediction_images_1.reshape(
    prediction_images_1.shape[0],
    7*7*512)

    prediction_images_2 = prediction_images_2.reshape(
    prediction_images_2.shape[0],
    5*5*2048)
    
    prediction_images_3 = prediction_images_3.reshape(
    prediction_images_3.shape[0],
    7*7*2048)

    # predicting tags for each array
    prediction_1 = np.argmax(model_1.predict(prediction_images_1), axis=1)
    prediction_2 = np.argmax(model_2.predict(prediction_images_2), axis=1)
    prediction_3 = np.argmax(model_3.predict(prediction_images_3), axis=1)

    # appending the mode of predictions in predict lists to assign the
    # tag to the video,
    # do the same for each model
    predict_1.append(y.columns.values[s.mode(prediction_1)[0][0]])
    predict_2.append(y.columns.values[s.mode(prediction_2)[0][0]])
    predict_3.append(y.columns.values[s.mode(prediction_3)[0][0]])

    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])

#prediction scores from each model
accuracy_score(predict_1, actual)*100
accuracy_score(predict_2, actual)*100
accuracy_score(predict_3, actual)*100